In [ ]:
#Configurando a API key localmente do Google Gemini
import os
os.environ.get('GOOGLE_API_KEY')

In [ ]:
# Configurando o cliente da SDK do Gemini 
from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunte ao Gemini uma informação mais recente que seu conhecimento 
from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(model=MODEL_ID,contents="Quando é a proxima imersão IA com Google Gemini da Alura?")

#Exibe a resposta na tela 
display(Markdown(f"Resposta:\n {resposta.text}"))

In [ ]:
# Pergunte ao Gemini uma informação utilizando a busca do Google como contexto 

response = client.models.generate_content(
    model=MODEL_ID, 
    contents='Quando é a proxima imersão IA com Google Gemini da Alura?', 
    config={"tools":[{"google_search": {}}]}
)

#Exibe a resposta na tela 
display(Markdown(f"Resposta:\n {resposta.text}"))
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

In [ ]:
# Instalar Framework ADK de agentes do Google ##################################
# pip install -q adk 

In [ ]:
from google.adk.agents import Agent # Realiza a abstração do agent 
from google.adk.runners import Runner # O orquestrador que ira "rodar tudo"
from google.adk.sessions import InMemorySessionService # A memoria interna do orquestrador 
from google.adk.tools import google_search # a tooll no caso a busca do google 
from google.genai import types #Para criar conteúdos (Content e Part)
from google import genai
from datetime import date
import textwrap #Para formatar melhor a saída de texto 
from IPython.display import display, Markdown #Para exibit texto formatado no colab 
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")


In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico,data_atual):
    buscador = Agent(
        name="agente_buscador",
        model= "gemini-2.0-flash",
        description="Agente que busca informações do Google", 
        tools=[google_search],
        instruction="""
        Você é um assistente de pesquisa. A sua rarefa é usar a ferramento de busca do google (google_search).
        Para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo. 
        Foque no máximo 5 lançamento relevantes, com base na quantidade e entusiamo das notícias sobre eles. 
        Se um tema tiver poucas notícias ou poucas reações entusiasmadas, é possível que ele não seja tão relevante assim 
        e podera ser substituído por outro que tenha mais reações ou noticias correlatas. 
        Esses lançamentos relevantes devem ser atuais, e de no máximo um mês antes da data de hoje. 
        Lembre-se o resultado de sua pesquisa deverá ser limitado a no máximo 5 nóticias apenas
        """
     )
    
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_atual}"
    lancamentos = call_agent(buscador,entrada_do_agente_buscador)
    return lancamentos

In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador", 
        model="gemini-2.0-flash", 
        description="Agente que planeja posts", 
        tools=[google_search],
        instruction="""
        Você é um assistente de planejador de conteúdo, especialista em redes sociais.
        Com base na lista de lançamentos mais recentese relevantes, você deve:
        Usar a ferramenta de busca do Google (google_search), para criar um plano sobre,
        quais são os pontos mais relevantes, em que poderíamos, abordar em um post sobre cada um deles.
        você também pode usar o (google_search) para encontrar mais informações sobre os temas e aprofundar mais sobre. 
        Ao final, você irá  escolher o tema mais relevante entre eles com base nas suas pesquisas e retornar esse tema, 
        seus pontos mais relevantes, e um plano com os assuntos a serem abordados no post que será escrito posteriormente. 
        """
    )
    
    entrada_do_agente_planejador = f"Tópico: {topico}\nLançamentos buscados{lancamentos_buscados}"
    #Executa o agente 
    plano_do_post = call_agent(planejador,entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
################################################
# --- Agente 3: Retador do Post --- #
################################################
def agente_redator(topico,plano_de_post):
    redator = Agent(
        name="agente_redator", 
        model="gemini-2.0-flash",
        description="Agente redator de posts engajadores para instagram ",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
        """
    )
    
    entrada_do_agente_retador = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    #Executa o agente 
    rascunho = call_agent(redator,entrada_do_agente_retador)
    return rascunho

In [ ]:
################################################
# --- Agente 4: Revisador de Qualidade --- #
################################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        description="Agente revisor de post para redes sociais.",
        instruction="""Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
        """
    )

    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor,entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")